In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import pickle

path_data= 'C:/root/Universita/project_dmdv/data/'

In [2]:
#funzione per estrarre la pagina
#decodificata in utf8, si vede dal tag meta

def get_html_tree(url):
    
    response = requests.get(url)
    html_page = response.content.decode('utf8','replace')#.text
    html_tree = BeautifulSoup(html_page)
    
    return html_tree

In [3]:
#estraggo la pagina base
url_base = 'https://www.emmys.com/awards/nominees-winners/2019/outstanding-comedy-series'
tree_base = get_html_tree(url_base)

#creo una lista con tutti gli url relativi ad ogni singolo premio
all_tags_ammyList = tree_base.find_all('div', class_ = 'col-12 col-md-4 col-lg-2')
award_links = []
for divs in all_tags_ammyList:
    award_links = award_links + [[divs.b.text, a_tags.text, 'https://www.emmys.com' + a_tags['href']] for a_tags in divs.find_all('a')]
data_links = pd.DataFrame(award_links, columns = ['group', 'award', 'link'])

In [4]:
l = []
for i in range(0, data_links.shape[0]):
    
    link = data_links.iloc[i]['link']
    base_html_tree = get_html_tree(link)
    print(data_links.iloc[i]['group'] + data_links.iloc[i]['award'])
    #estrazione di tutti gli anni associati al premio
    all_years = [a_tags['href'][-4:] for a_tags in base_html_tree.find_all('div', class_ = 'search-links')[0].find_all('a')]
    
    #ciclo su tutti gli anni
    for year in all_years:
        
        #creazione della stringa con link dell'anno corrente
        current_link = link.split('/')
        current_link[-2] = year
        current_link = '/'.join(current_link)
        
        #albero della pagina da ispezionare
        year_html_tree = get_html_tree(current_link)
        
        #lista di record associato ad ogni nominato
        all_nominee = year_html_tree.find_all('div', class_ = 'item')
        for item in all_nominee:
            
            is_winner = 'Winner' if 'winner' in item.attrs['class'] else 'Nominee'
            
            #lista lunga 2 se è un premio ad un attore/attrice, lunga 1 altrimenti (il titolo è all'ultima posizione)
            text_title = item.find_all('div', class_ = 'title')[0].find_all('a')
            title = text_title[-1].text
            people = []
            
            #se e premio per attore text title e' lungo 2 e assegno il nome della persona escludendo
            #l'ultimo elemento che contiene il titolo della serie, altrimenti uso il tag description per
            #recuperare direttori/scrittori (se e' il premio principale seguire il commento sotto)
            if len(text_title) > 1:
                people = [t.text for t in text_title[:-1]]
            else:
                
                #se nel premio principale non si vuole mettere niente va tolto questo else e 
                #il codice dentro all'else va messo nel seguente if per poter assegnare il premio
                #per scrittura e direzione alle persone
                #if ('DIRECTING' in upper(data_links.iloc[i]['award'])) | ('WRITING' in upper(data_links.iloc[i]['award'])):
                if len(item.find_all('div', class_ = 'description')) > 0:
                    people = [p.text for p in item.find_all('div', class_ = 'description')[0].find_all('a')]
                    if len(people) == 0:
                        people = [re.sub(r',\s.*', '', p.text) for p in item.find_all('div', class_ = 'description')[0].find_all('li')]
                else:
                    people = [re.sub(r'\sas\s.*', '', p.text) for p in  item.find_all('p')] if len(item.find_all('p')[0]) > 0 else []
            
            current_data = [data_links.iloc[i]['group'] + ' - ' + data_links.iloc[i]['award'], float(year), title,  people,  is_winner]
            l = l + [current_data]


ComedyComedy Series
ComedyLead Actor
ComedyLead Actress
ComedySupporting Actor
ComedySupporting Actress
ComedyGuest Actor
ComedyGuest Actress
ComedyDirecting
ComedyWriting
DramaDrama Series
DramaLead Actor
DramaLead Actress
DramaSupporting Actor
DramaSupporting Actress
DramaGuest Actor
DramaGuest Actress
DramaDirecting
DramaWriting
Limited Series, MovieLimited Series
Limited Series, MovieTelevision Movie
Limited Series, MovieLead Actor
Limited Series, MovieLead Actress
Limited Series, MovieSupporting Actor
Limited Series, MovieSupporting Actress
Limited Series, MovieDirecting
Limited Series, MovieWriting
RealityHost For A Reality Or Competition Program
RealityStructured Program
RealityUnstructured Program
RealityCompetition Program
RealityDirecting
Short FormComedy Or Drama Series
Short FormActor
Short FormActress
Short FormNonfiction Or Reality Series
Short FormVariety Series
VarietyVariety Talk Series
VarietyVariety Sketch Series
VarietyVariety Special (Live)
VarietyVariety Special (

In [6]:
final_data = pd.DataFrame(l, columns = ['award', 'year', 'program', 'person', 'is_winner'])
final_data.head()

,award,year,program,person,is_winner
0,Comedy - Comedy Series,2019.0,Barry,"[Alec Berg, Bill Hader, Aida Rodgers, Emily He...",Nominee
1,Comedy - Comedy Series,2019.0,Fleabag,"[Phoebe Waller-Bridge, Harry Bradbeer, Lydia H...",Winner
2,Comedy - Comedy Series,2019.0,Russian Doll,"[Natasha Lyonne, Leslye Headland, Amy Poehler,...",Nominee
3,Comedy - Comedy Series,2019.0,Schitt's Creek,"[Eugene Levy, Daniel Levy, David West Read, An...",Nominee
4,Comedy - Comedy Series,2019.0,The Good Place,"[Michael Schur, David Miner, Morgan Sackett, D...",Nominee


In [7]:
final_data.to_pickle(path_data + 'awards/emmy_data.dataset')